In [21]:
! python3 -m pip install --upgrade pip
! python3 -m pip install  --upgrade SpeechRecognition
! python3 -m pip install  --upgrade pydub

In [63]:
import tensorflow as tf
import librosa
import kapre
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd
import pandas as pd
import numpy as np
import pathlib
import os
import re
import speech_recognition as sr
from os import listdir
from os.path import isfile, join
from os import path
from plotnine import *
from pydub import AudioSegment

# Transcribe audio data to txt data

In [22]:
# TODO make an input from user
data_path = '/tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train'

audio_path_cc = data_path + '/Full_wave_enhanced_audio/cc/'
audio_path_cd = data_path + '/Full_wave_enhanced_audio/cd/'


text_path_cc = data_path + '/transcription/cc/'
text_path_cd = data_path + '/transcription/cd/'

lang_ = 'en-US'

r = sr.Recognizer()

In [59]:
def startConversion(path, filename, lang = lang_):
                
    # Create output file name
    output_dir = path + 'output_text/'
    output_file = '.'.join(filename.split(sep='.')[:-1]) + '.txt'
    output_file_path = output_dir + output_file
    
    # If output file does not exist, continue
    if os.path.exists(output_test_file_path):
        print("Sorry, " + output_test_file_path + " already exists")
    else:
        full_path = path + filename

        with sr.AudioFile(full_path) as source:
            print('Transcribing file: ' + str(filename) + ' in path: ' + str(full_path))
            audio_text = r.listen(source)
            # recognize_() method will throw a request error if the API is unreachable, hence using exception handling
            try:

                # using google speech recognition
                # print('Converting audio transcripts into text ...')
                text = r.recognize_google(audio_text)

                # Create output directory
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)

                with open(output_file_path, 'w') as f:
                    f.write(text)
                print('Finished transcribing text file ' + str(output_file) + ' at location ' + output_file_path)

            except Exception as e:
                print('Error: ' + str(e) + ' <- this guy')

In [55]:
# Running for CC path
onlyfiles = [f for f in listdir(audio_path_cc) if isfile(join(audio_path_cc, f))]

for filename in onlyfiles:
    startConversion(path = audio_path_cc, filename = filename)

Filename: S001.wav in path: /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/S001.wav
Finished transcribing text file S001.txtat location /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/output_text/S001.txt
Filename: S002.wav in path: /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/S002.wav
Finished transcribing text file S002.txtat location /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/output_text/S002.txt
Filename: S003.wav in path: /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/S003.wav
Finished transcribing text file S003.txtat location /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/output_text/S003.txt
Filename: S004.wav in path: /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_

Finished transcribing text file S038.txtat location /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/output_text/S038.txt
Filename: S039.wav in path: /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/S039.wav
Finished transcribing text file S039.txtat location /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/output_text/S039.txt
Filename: S040.wav in path: /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/S040.wav
Finished transcribing text file S040.txtat location /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/output_text/S040.txt
Filename: S041.wav in path: /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/S041.wav
Finished transcribing text file S041.txtat location /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/t

In [56]:
# Running for CD path
onlyfiles = [f for f in listdir(audio_path_cd) if isfile(join(audio_path_cd, f))]

for filename in onlyfiles:
    startConversion(path = audio_path_cd, filename = filename)

Filename: S079.wav in path: /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cd/S079.wav
Finished transcribing text file S079.txtat location /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cd/output_text/S079.txt
Filename: S080.wav in path: /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cd/S080.wav
Finished transcribing text file S080.txtat location /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cd/output_text/S080.txt
Filename: S081.wav in path: /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cd/S081.wav
Finished transcribing text file S081.txtat location /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cd/output_text/S081.txt
Filename: S082.wav in path: /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_

Error:  <- this guy
Filename: S126.wav in path: /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cd/S126.wav
Finished transcribing text file S126.txtat location /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cd/output_text/S126.txt
Filename: S127.wav in path: /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cd/S127.wav
Finished transcribing text file S127.txtat location /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cd/output_text/S127.txt
Filename: S128.wav in path: /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cd/S128.wav
Finished transcribing text file S128.txtat location /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cd/output_text/S128.txt
Filename: S129.wav in path: /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train

# EDA

In [66]:
# Running for CC path
audio_cc_text_path = audio_path_cc + 'output_text/'
onlytextfiles = [f for f in listdir(audio_cc_text_path) if isfile(join(audio_cc_text_path, f))]

# Create output directory
audio_cc_clean_text_path_dir = audio_cc_text_path + 'output_text_clean/'
if not os.path.exists(audio_cc_clean_text_path_dir):
    os.makedirs(audio_cc_clean_text_path_dir)

for filename in onlytextfiles:
    output_clean_text_file_path = audio_cc_clean_text_path_dir + filename
    if os.path.exists(output_clean_text_file_path):
        print("Sorry, " + output_test_file_path + " already exists")
    else:    
        print("Cleaning up: " + str(filename))
        full_text_file_path = audio_cc_text_path + filename

        # Make all text lowercase
        # Remove special characters
        file = open(full_text_file_path, 'r')

        lines = [re.sub('[^a-zA-Z0-9\n\.]','',line.lower()) for line in file]    
        with open(output_clean_text_file_path, 'w') as out:
             out.writelines(sorted(lines))
print("Done!")

Sorry, /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/output_text/S001.txt already exists
Sorry, /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/output_text/S001.txt already exists
Sorry, /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/output_text/S001.txt already exists
Sorry, /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/output_text/S001.txt already exists
Sorry, /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/output_text/S001.txt already exists
Sorry, /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/output_text/S001.txt already exists
Sorry, /tf/dementia/0extra/ADReSS-IS2020-train/ADReSS-IS2020-data/train/Full_wave_enhanced_audio/cc/output_text/S001.txt already exists
Sorry, /tf/dementia/0extra/ADReSS-IS2020-train/A

In [68]:
# Running for CD path
audio_cd_text_path = audio_path_cd + 'output_text/'
onlytextfiles = [f for f in listdir(audio_cd_text_path) if isfile(join(audio_cd_text_path, f))]

# Create output directory
audio_cd_clean_text_path_dir = audio_cd_text_path + 'output_text_clean/'
if not os.path.exists(audio_cd_clean_text_path_dir):
    os.makedirs(audio_cd_clean_text_path_dir)

for filename in onlytextfiles:
    output_clean_text_file_path = audio_cd_clean_text_path_dir + filename
    if os.path.exists(output_clean_text_file_path):
        print("Sorry, " + output_test_file_path + " already exists")
    else:     
        print("Cleaning up: " + str(filename))
        full_text_file_path = audio_cd_text_path + filename

        # Make all text lowercase
        # Remove special characters
        file = open(full_text_file_path, 'r')

        lines = [re.sub('[^a-zA-Z0-9\n\.]','',line.lower()) for line in file]
        with open(output_clean_text_file_path, 'w') as out:
             out.writelines(sorted(lines))
print("Done!")

Cleaning up: S079.txt
Cleaning up: S080.txt
Cleaning up: S081.txt
Cleaning up: S082.txt
Cleaning up: S083.txt
Cleaning up: S084.txt
Cleaning up: S086.txt
Cleaning up: S087.txt
Cleaning up: S090.txt
Cleaning up: S092.txt
Cleaning up: S093.txt
Cleaning up: S094.txt
Cleaning up: S095.txt
Cleaning up: S096.txt
Cleaning up: S097.txt
Cleaning up: S100.txt
Cleaning up: S103.txt
Cleaning up: S104.txt
Cleaning up: S107.txt
Cleaning up: S108.txt
Cleaning up: S110.txt
Cleaning up: S111.txt
Cleaning up: S114.txt
Cleaning up: S116.txt
Cleaning up: S118.txt
Cleaning up: S122.txt
Cleaning up: S126.txt
Cleaning up: S127.txt
Cleaning up: S128.txt
Cleaning up: S129.txt
Cleaning up: S130.txt
Cleaning up: S132.txt
Cleaning up: S135.txt
Cleaning up: S136.txt
Cleaning up: S137.txt
Cleaning up: S138.txt
Cleaning up: S139.txt
Cleaning up: S140.txt
Cleaning up: S141.txt
Cleaning up: S142.txt
Cleaning up: S143.txt
Cleaning up: S144.txt
Cleaning up: S145.txt
Cleaning up: S148.txt
Cleaning up: S149.txt
Cleaning u

# Split dataset

# Train GMM